This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace.


# 🧙‍♂️ HomeMatch: Magical Real Estate Finder

This notebook implements HomeMatch, an innovative real estate application that uses advanced AI techniques to personalize property searches for wizards and witches in the magical world. The project leverages Large Language Models (LLMs) and vector databases to transform standard listings into enchanting, personalized narratives.

### Key Components
1. **Synthetic Data Generation**: Using GPT-4o-mini to create diverse, magical real estate listings.
2. **Vector Database Integration**: Employing ChromaDB to store and efficiently search property embeddings.
3. **Semantic Search**: Implementing preference-based searches to find the most suitable magical dwellings.
4. **Personalized Descriptions**: Utilizing LLMs to craft tailored property descriptions that resonate with each wizard's unique needs.
5. **PoC of a Interactive Web Interface**: Building a user-friendly web application to showcase the magic of HomeMatch.

In [1]:
# Requirements
%pip install -q python-dotenv langchain openai chromadb==0.5.3 tenacity

Note: you may need to restart the kernel to use updated packages.


In [2]:
import dotenv
dotenv.load_dotenv() # Load environment variables from .env file (OpenAI API key) 

True

In [3]:
from langchain_openai import ChatOpenAI
#from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
#from langchain_community.vectorstores import Chroma
#from langchain_openai import OpenAIEmbeddings
#from langchain.schema import Document

chat_model = "gpt-4o-mini"
temperature = 0.7

# Initialize LLM
llm = ChatOpenAI(model_name=chat_model, temperature=temperature)

## Component 1: Synthetic Data Generation

This component focuses on creating diverse and realistic magical city contexts and real estate listings using Large Language Models (LLMs). 

### 1.1 City Context Generation

We use GPT-4o-mini to generate unique city contexts for our magical real estate listings. Each context includes:
1. A creative neighborhood name
2. General location or region
3. Population size category
4. Defining characteristics or attractions
5. Economic base or major industries

These contexts serve as the foundation for our property listings, ensuring a rich and diverse magical world.

#### Code Overview

The `generate_city_contexts` function uses a carefully crafted prompt to generate city contexts. We then save these contexts to a CSV file for persistence and easy reuse.

Key functions:
- `generate_city_contexts`: Creates city contexts using the LLM
- `save_city_contexts_to_csv`: Saves or appends contexts to a CSV file
- `load_city_contexts_from_csv`: Loads previously generated contexts

In [4]:
import pandas as pd
import os
from typing import List
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


def generate_city_contexts(llm, num_contexts: int = 10) -> List[str]:
    """
    Generate a specified number of city context descriptions.

    Args:
    num_contexts (int): Number of city contexts to generate. Defaults to 10.

    Returns:
    List[str]: List of generated city context descriptions.
    """

    city_context_prompt = PromptTemplate.from_template(
        """Generate a very short, realistic but, completely made-up, context description for a city or neighborhood that could be used in real estate listings for the Harry Potter series. Include:
        1. Neighborhood name (be creative and diverse)
        2. General location or region
        3. Population size category (small town, mid-size city, large metropolis, etc.)
        4. One or two defining characteristics or attractions
        5. Economic base or major industries

        Provide this in a concise paragraph format, suitable for use in generating real estate listings.

        Example:
        Zenithville Dwarf Parking, nestled in the heart of the Purple Pot Midwest, is a charming mid-size city of about 150,000 broom stick students. Known for its vibrant magic arts scene and annual hot air soap bubble festival, Zenithville has recently become a hub for fairy tech wizards, blending its traditional wizardry schools with a growing innovation sector.

        Now, generate a fantastic unique new city context:"""
    )

    contexts = []
    for _ in range(num_contexts):
        context = llm.invoke(city_context_prompt.format())
        contexts.append(context.content.strip())

    return pd.DataFrame({"city_context": contexts})


def save_city_contexts_to_csv(
    df_city_contexts: pd.DataFrame,
    append: bool = False,
    city_contexts_csv: str = "homematch_city_contexts.csv",
):
    """
    Generate and save city contexts to a CSV file.

    Args:
    city_contexts_csv (str): Path to the CSV file to save the city contexts. Defaults to "city_contexts.csv".
    """

    # if the csv does not exists, create it
    if not os.path.exists(city_contexts_csv):

        # Save to CSV
        df_city_contexts.to_csv(city_contexts_csv, index=False)
        print(f"City contexts saved to {city_contexts_csv}")

    else:
        if append:
            # Load the existing CSV
            existing_df = pd.read_csv(city_contexts_csv)
            # Append the new data
            df_city_contexts = pd.concat(
                [existing_df, df_city_contexts], ignore_index=True
            )
            # Save to CSV
            df_city_contexts.to_csv(city_contexts_csv, index=False)
            print(f"City contexts appended to {city_contexts_csv}")
        else:
            print(
                f"City contexts already exist in {city_contexts_csv}. Set 'append=True' to append the new contexts."
            )


def load_city_contexts_from_csv(
    city_contexts_csv: str = "homematch_city_contexts.csv",
) -> pd.DataFrame:
    """
    Load city contexts from a CSV file.

    Args:
    city_contexts_csv (str): Path to the CSV file containing the city contexts. Defaults to "city_contexts.csv".

    Returns:
    pd.DataFrame: DataFrame containing the loaded city contexts.
    """
    if os.path.exists(city_contexts_csv):
        df_city_contexts = pd.read_csv(city_contexts_csv)
        print(f"City contexts loaded from {city_contexts_csv}")
        return df_city_contexts
    else:
        print(f"City contexts CSV file '{city_contexts_csv}' not found.")
        return None

In [ ]:
# Generate city contexts
df = generate_city_contexts(llm=llm)  # Generate n=10 city contexts
save_city_contexts_to_csv(df, True)

In [5]:
# Load city contexts from CSV file
df = load_city_contexts_from_csv()

# Display the city contexts DataFrame
df.tail()

City contexts loaded from homematch_city_contexts.csv


,city_context
10,"Wandelo, located in the lush hills of the Ench..."
11,"Gangrove, located in the enchanting Emerald Va..."
12,"Elderbore, nestled in the mystical Glimmering ..."
13,"Princebeek, located within the enchanting Elde..."
14,"Manickure City, located in the lush and mystic..."


#### Example City Context Generation

Brackenvale Hollow, located in the serene outskirts of the Enchanted Forest region, is a quaint small town with a population of around 8,000 magical beings. Renowned for its breathtaking views of the Whispering Pines and the enchanting Glowworm Festival held every autumn, Brackenvale offers a unique blend of tranquility and community spirit. The town's economy thrives on potion brewing, magical creature care, and artisanal wand craftsmanship, making it a perfect haven for both seasoned wizards and aspiring apprentices looking to settle down in a picturesque, magical environment.


In [6]:
from pydantic import BaseModel, Field, StrictStr, NonNegativeInt
from langchain.output_parsers import PydanticOutputParser


class RealEstateListing(BaseModel):
    neighborhood: StrictStr = Field(description="name of the neighborhood")
    neighborhood_description: StrictStr = Field(
        description="description of the neighborhood"
    )
    price: NonNegativeInt = Field(description="price of the listing")
    bedrooms: NonNegativeInt = Field(description="number of bedrooms")
    bathrooms: NonNegativeInt = Field(description="number of bathrooms")
    sqft: NonNegativeInt = Field(description="square footage of the listing")
    description: StrictStr = Field(description="description of the listing")


parser = PydanticOutputParser(pydantic_object=RealEstateListing)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"neighborhood": {"description": "name of the neighborhood", "title": "Neighborhood", "type": "string"}, "neighborhood_description": {"description": "description of the neighborhood", "title": "Neighborhood Description", "type": "string"}, "price": {"description": "price of the listing", "minimum": 0, "title": "Price", "type": "integer"}, "bedrooms": {"description": "number of bedrooms", "minimum": 0, "title": "Bedrooms", "type": "integer"}, "bathrooms": {"description": "number of bathrooms", "minimum": 0, "title": "Bathrooms", 

In [7]:
prompt = PromptTemplate(
    template="{question}\n\nFormat requirements:\n{format_instructions}\n\nCity Context:\n{context}",
    input_variables=["question", "context"],
    partial_variables={"format_instructions": parser.get_format_instructions},
)

question = """Generate a fantasy real estate listing like it's from the Harry Potter series with the following details:
		- Neighborhood
		- Neighborhood Description
		- Price
		- Bedrooms
		- Bathrooms
		- House Size
		- Description
		
		Make it diverse and appealing. Please sum up these details in the description as well!"""


# Generate a real estate listing prompt with a random city context from the DataFrame
for city_context in df["city_context"].sample(10):
    query = prompt.format(context=city_context, question=question)

print(query)

Generate a fantasy real estate listing like it's from the Harry Potter series with the following details:
		- Neighborhood
		- Neighborhood Description
		- Price
		- Bedrooms
		- Bathrooms
		- House Size
		- Description
		
		Make it diverse and appealing. Please sum up these details in the description as well!

Format requirements:
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"neighborhood": {"description": "name of the neighborhood", "title": "Neighborhood", "type": "string"}, "neighborhood_description": {"description": "description of the neighborhood", "title": "Neighb

In [8]:
response = llm.invoke(query)
print(response)

content='```json\n{\n  "neighborhood": "Wyrmwood",\n  "neighborhood_description": "Nestled in the enchanting Elderwood region, Wyrmwood is a quaint small town known for its lush, whispering forests and the magical Lantern Festival. With a diverse population of approximately 8,500 magical beings, this picturesque community blends serene nature with vibrant tradition, thriving on potion brewing, wand crafting, and eco-friendly magical tourism.",\n  "price": 350000,\n  "bedrooms": 3,\n  "bathrooms": 2,\n  "sqft": 1600,\n  "description": "Step into this charming 3-bedroom, 2-bathroom cottage, a perfect blend of rustic charm and modern comforts. The spacious 1600 sqft layout features enchanted windows that open to reveal breathtaking views of the surrounding Elderwood trees, while the cozy fireplace crackles with a flicker of magic. The open-plan living area is ideal for gatherings, whether for potion-making sessions or enjoying festive evenings during the Lantern Festival. The lush garden,

In [9]:
result = parser.parse(response.content)
print(result.dict())

{'neighborhood': 'Wyrmwood', 'neighborhood_description': 'Nestled in the enchanting Elderwood region, Wyrmwood is a quaint small town known for its lush, whispering forests and the magical Lantern Festival. With a diverse population of approximately 8,500 magical beings, this picturesque community blends serene nature with vibrant tradition, thriving on potion brewing, wand crafting, and eco-friendly magical tourism.', 'price': 350000, 'bedrooms': 3, 'bathrooms': 2, 'sqft': 1600, 'description': 'Step into this charming 3-bedroom, 2-bathroom cottage, a perfect blend of rustic charm and modern comforts. The spacious 1600 sqft layout features enchanted windows that open to reveal breathtaking views of the surrounding Elderwood trees, while the cozy fireplace crackles with a flicker of magic. The open-plan living area is ideal for gatherings, whether for potion-making sessions or enjoying festive evenings during the Lantern Festival. The lush garden, complete with a herb patch and a small 

### 1.2 Synthetic Listing Generation

After generating city contexts, we create detailed magical real estate listings for each city. This process involves using our LLM to generate rich, fantastical property descriptions that align with the unique characteristics of each magical location.

#### Listing Generation Process

1. **Single Listing Generation**: We use a carefully crafted prompt to generate individual listings, ensuring each property has unique features that fit the magical world.

2. **Multiple Listings**: We generate multiple listings for each city context, creating a diverse range of properties within each magical location.

3. **Error Handling and Retries**: To ensure robustness, we implement retry logic for listing generation, handling potential API errors or rate limiting issues.

4. **Persistence**: Generated listings are saved to a JSON file, allowing for easy storage and retrieval of our magical property database.

#### Key Functions

- `generate_single_listing`: Creates a single property listing using the LLM.
- `generate_multiple_listings`: Generates multiple listings across different city contexts.
- `save_listings_to_file`: Persists the generated listings to a JSON file.

This approach allows us to create a rich, diverse database of magical properties that forms the foundation of our HomeMatch application.

In [10]:
import json
from typing import List, Dict
from tenacity import retry, stop_after_attempt, wait_random_exponential


@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(3))
def generate_single_listing(llm, prompt, parser, city_context):
    """Generate a single real estate listing with error handling and retries."""
    try:
        query = prompt.format(context=city_context, question=question)
        response = llm.invoke(query)
        result = parser.parse(response.content)
        return result.dict()
    except Exception as e:
        print(f"Error generating listing: {e}")
        raise


def generate_multiple_listings(llm, prompt, parser, df_city_contexts, num_listings=10):
    """Generate multiple real estate listings."""
    listings = []
    for i in range(num_listings):
        city_context = df_city_contexts["city_context"].iloc[i]
        try:
            listing = generate_single_listing(llm, prompt, parser, city_context)
            listings.append(listing)
        except Exception as e:
            print(f"Failed to generate listing: {e}")
    return listings


def save_listings_to_file(
    listings: List[Dict],
    append: bool = False,
    filename: str = "homematch_listings.json",
):
    """Save generated listings to a JSON file with an option to append."""
    try:
        if append and os.path.exists(filename):
            with open(filename, "r") as f:
                existing_data = json.load(f)
            existing_data.extend(listings)
            data_to_save = existing_data
        else:
            data_to_save = {"listings": listings}

        with open(filename, "w") as f:
            json.dump(data_to_save, f, indent=2)
        print(f"{'Appended to' if append else 'Saved'} {filename}")
    except Exception as e:
        print(f"Error saving listings to file: {e}")

In [34]:
# Generate listings
listings = generate_multiple_listings(llm, prompt, parser, df)

In [35]:
# Safe listings to file
save_listings_to_file(listings, append=True)  # Set append=True to add to existing file

Appended to homematch_listings.json


In [11]:
import json
from typing import List, Dict
from tenacity import retry, stop_after_attempt, wait_random_exponential


def load_listings_from_file(filename: str = "homematch_listings.json") -> List[Dict]:
    """Load listings from a JSON file."""
    try:
        with open(filename, "r") as f:
            data = json.load(f)
            print(f"Loaded {len(data.get('listings', []))} listings from {filename}")
        return data.get("listings", [])
    except FileNotFoundError:
        print(f"File {filename} not found. Returning empty list.")
        return []
    except json.JSONDecodeError:
        print(f"Error decoding JSON from {filename}. Returning empty list.")
        return []
    except Exception as e:
        print(f"Error loading listings from file: {e}")
        return []

In [12]:
# Load listings from file
loaded_listings = load_listings_from_file()
loaded_listings[:3]  # Display the first 3 listings

Loaded 10 listings from homematch_listings.json


[{'neighborhood': 'Brackenvale',
  'neighborhood_description': 'Brackenvale is a quaint small town nestled on the serene outskirts of the Enchanted Forest. With a population of around 8,000 magical beings, it boasts breathtaking views of the Whispering Pines and hosts the enchanting Glowworm Festival every autumn. Known for its strong community spirit and thriving economy based on potion brewing, magical creature care, and artisanal wand craftsmanship, Brackenvale is the perfect haven for both seasoned wizards and aspiring apprentices.',
  'price': 355000,
  'bedrooms': 3,
  'bathrooms': 2,
  'sqft': 1800,
  'description': 'Welcome to your dream home in the heart of Brackenvale! This charming 3-bedroom, 2-bathroom cottage offers 1,800 sqft of magical living space. Featuring an open-concept layout with a cozy fireplace that crackles with enchantment, a spacious kitchen perfect for potion brewing, and a garden filled with rare herbs and flowers, this property is a true gem. The master su

## Component 2: Vector Database Integration

This component focuses on integrating our generated listings with a vector database, allowing for efficient semantic search capabilities.

### Vector Store Setup

We use Chroma as our vector database, which allows us to store and search our listings based on their semantic meaning rather than just keywords. This process involves:

1. **Initialization**: Setting up a Chroma client with persistent storage.
2. **Collection Creation**: Creating a collection to store our listings.
3. **Document Preparation**: Transforming our listings into a format suitable for the vector store.
4. **Embedding Generation**: Using OpenAI's embedding model to create vector representations of our listings.
5. **Data Insertion**: Adding the embedded documents and their metadata to the Chroma collection.

This setup enables us to perform semantic searches on our listings, matching properties to user preferences based on the overall meaning and context rather than exact word matches. It forms the backbone of our semantic search capabilities in HomeMatch.

In [12]:
import chromadb
from chromadb.config import Settings
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document

# Initialize the Chroma client
client = chromadb.PersistentClient(path="chroma.db", settings=Settings())

# Create a collection
collection_name = "homematch_listings_store"
collection = client.get_or_create_collection(name=collection_name)

# Prepare the documents and embeddings
embeddings = OpenAIEmbeddings()
documents = []
ids = []
metadatas = []

for i, listing in enumerate(loaded_listings):
    page_content = f"{listing['description']} {listing['neighborhood_description']}"
    metadata = listing.copy()
    del metadata["description"]
    del metadata["neighborhood_description"]

    documents.append(page_content)
    ids.append(str(i))
    metadatas.append(metadata)

# Generate embeddings
embedded_documents = embeddings.embed_documents(documents)

# Add documents to the collection
collection.upsert(
    embeddings=embedded_documents, documents=documents, metadatas=metadatas, ids=ids
)

In [56]:
# vectorstore.delete_collection() # for debugging purposes only

## Component 3: Semantic Search

The Semantic Search component is at the heart of HomeMatch's ability to match magical properties with wizard preferences. This feature allows us to go beyond simple keyword matching, understanding the context and meaning behind a user's search query.

### How it Works

1. **Query Embedding**: When a user inputs their preferences, we convert this natural language query into a vector embedding using the same model we used for our listings.

2. **Similarity Matching**: We compare this query embedding against all the property embeddings in our vector store, finding the most semantically similar listings.

3. **Relevance Scoring**: Each potential match is assigned a relevance score, indicating how closely it aligns with the user's preferences.

4. **Result Retrieval**: We retrieve the top-k most relevant listings based on these similarity scores.

### Key Features

- Natural language understanding of user preferences
- Context-aware matching of properties to user needs
- Ability to find relevant listings even when exact keywords don't match
- Customizable number of results returned

This semantic search capability allows HomeMatch to provide highly personalized and relevant property suggestions, even for complex or nuanced user requirements.

In [17]:
# Create a Langchain wrapper around the Chroma collection
import chromadb
from chromadb.config import Settings
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document

collection_name = "homematch_listings_store"

# Initialize the Chroma client
client = chromadb.PersistentClient(path="chroma.db", settings=Settings())

embeddings = OpenAIEmbeddings()

vectorstore = Chroma(
    client=client,
    collection_name=collection_name,
    embedding_function=embeddings,
    persist_directory="chroma.db",
)

In [18]:
vectorstore.get(["0", "1", "2"])

{'ids': ['0', '1', '2'],
 'embeddings': None,
 'metadatas': [{'bathrooms': 2,
   'bedrooms': 3,
   'neighborhood': 'Brackenvale',
   'price': 355000,
   'sqft': 1800},
  {'bathrooms': 2,
   'bedrooms': 3,
   'neighborhood': 'Silvershadow',
   'price': 370000,
   'sqft': 1500},
  {'bathrooms': 2,
   'bedrooms': 3,
   'neighborhood': 'Wyrmwood',
   'price': 320000,
   'sqft': 1800}],
 'documents': ['Welcome to your dream home in the heart of Brackenvale! This charming 3-bedroom, 2-bathroom cottage offers 1,800 sqft of magical living space. Featuring an open-concept layout with a cozy fireplace that crackles with enchantment, a spacious kitchen perfect for potion brewing, and a garden filled with rare herbs and flowers, this property is a true gem. The master suite includes a whimsical balcony overlooking the Whispering Pines, while the guest rooms are perfect for visiting friends or family. With the Glowworm Festival just around the corner, you’ll be at the center of all the festivities.

In [19]:
# search for similar listings
query = "I'm looking for a home. Something like a cottage with two bathrooms and three bedrooms in the woods by a lake"
results = vectorstore.similarity_search_with_relevance_scores(query, k=3)
results

[(Document(page_content="Welcome to this charming 3-bedroom, 2-bathroom cottage located in the heart of Thornwick! With 1800 sqft of cozy living space, this home features enchanting stained glass windows that let in the soft glow of the forest light. The open-concept living area is perfect for gatherings, with a fireplace that crackles with a gentle warmth. The kitchen is a chef's delight, complete with a cauldron-compatible stove and ample space for potion-making. The master bedroom offers a serene view of the Whispering Woods, while the lush garden is ideal for growing your own herbs. Enjoy the vibrant community spirit with easy access to the Moonlight Market and local apothecaries. This property is not just a home; it's a magical lifestyle waiting for you! Thornwick is a quaint village nestled in the enchanting Whispering Woods region. This magical community of around 5,000 residents is celebrated for its picturesque cobblestone streets and the legendary Moonlight Market, attracting

In [20]:
filter_dict = {"price": 320000}  # Filter by price on metadata

vectorstore.similarity_search_with_relevance_scores(
    query="I'm looking for a two-story cottage and wand-friendly workspace with a single bathroom",
    k=5,
    filter=filter_dict,
)

[(Document(page_content='Welcome to your new magical haven in Wyrmwood! This charming 3-bedroom, 2-bathroom cottage spans 1,800 sqft and features enchanting forest views from every window. The cozy living room, adorned with a crackling fireplace and floating bookshelves, invites you to relax after a long day of potion brewing. The spacious kitchen is equipped with a cauldron-stove and enchanted cabinets that organize themselves. Step outside to a beautifully manicured garden, perfect for herb cultivation and hosting delightful gatherings during the Lantern Festival. With its blend of tranquility and craftsmanship, this home is perfect for families and magical artisans alike. Nestled in the enchanting Elderwood region, Wyrmwood is a quaint small town home to around 8,500 magical beings. Known for its lush, whispering forests and the captivating Lantern Festival, this picturesque community offers a unique blend of serene nature and vibrant tradition, ideal for those who appreciate a magi

## Component 4: Personalized Listing Description Generation

The Personalized Listing Description Generation component is where HomeMatch truly shines, transforming standard property listings into enchanting, tailored narratives that speak directly to each wizard's unique preferences and needs.

### How it Works

1. **User Preference Analysis**: We analyze the user's search query and any additional information provided to understand their specific desires and requirements.

2. **Listing Selection**: Using our semantic search results, we select the most relevant properties for the user.

3. **Context Merging**: We combine the original listing information with the user's preferences to create a context for personalization.

4. **LLM-Powered Rewriting**: Leveraging our Large Language Model (GPT-4o-mini), we generate a new, personalized description for each selected property.

5. **Fact Checking**: We ensure that all factual information about the property remains accurate, while the presentation is tailored to highlight aspects most relevant to the user.

### Key Features

- Highly personalized property descriptions
- Emphasis on features that align with user preferences
- Maintenance of factual accuracy
- Engaging, wizard-friendly language that brings each property to life

This component transforms the house-hunting experience from a mundane task into a magical journey, helping users envision themselves in each property and making emotional connections with potential homes.

In [21]:
# Define user preferences
def personalize_listings(listings_from_search, user_preferences_from_chat):
    prompt = PromptTemplate(
        input_variables=["listing", "preferences"],
        template="""
        You are a magical, but sneaky, realty sales person from the Harry Potter series. Use the following listings as inspiration to make an compelling offer of options to the user. The user has already been shown the listings, so it's your job to give selling arguments why these listings totally fit the user's preferences: {user_preferences_from_chat}
        
        Original listings: {listings_from_search}
        
        Eaxmple offering:
        Take a look at this Cottage in Thornwick. Imagine stepping into a **charming 3-bedroom** cottage nestled in the heart of **Thornwick**, where the whispers of the **Whispering Woods** beckon. While it boasts **two bathrooms**, the **cozy 1800 sqft** of living space is more than adequate for your needs. Picture yourself gathering around the **crackling fireplace** in the open-concept living area, while the **chef's kitchen** with its **cauldron-compatible stove** serves as the perfect backdrop for your potion-making artistry. The **lush garden** is not just a place for herbs; it's a magical sanctuary where you can cultivate your ingredients and let your creativity flourish. The community spirit here is vibrant, with the **Moonlight Market** just a wand's flick away. With its enchanting stained glass windows, this cottage is not just a home; it’s a lifestyle filled with magic and wonder!
        
        Now sell it to the user as skilfull as you possibly can!: 
        """
    )
    
    chain = LLMChain(llm=llm, prompt=prompt, verbose=True)
    personalized_description = chain.run(listings_from_search=listings_from_search, user_preferences_from_chat=user_preferences_from_chat)
    
    return personalized_description



In [22]:
# test the function with the search results and user preferences
user_preferences = "I'm looking for a two-story cottage and wand-friendly workspace with a single bathroom, can you help me with that?"

listings = []
for doc, score in results:
    listings.append(doc)

personalized_response = personalize_listings(listings_from_search=listings, user_preferences_from_chat=user_preferences)
print(personalized_response)

/Users/leonvanbokhorst/repos/HomeMatch/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:

        You are a magical, but sneaky, realty sales person from the Harry Potter series. Use the following listings as inspiration to make an compelling offer of options to the user. The user has already been shown the listings, so it's your job to give selling arguments why these listings totally fit the user's preferences: I'm looking for a two-story cottage and wand-friendly workspace with a single bathroom, can you help me with that?
        
        Original listings: [Document(page_content="Welcome to this charming 3-bedroom, 2-bathroom cottage located in the heart of Thornwick! With 1800 sqft of cozy living space, this home features enchanting stained glass windows that let in the soft glow of the forest light. The open-concept living area is perfect for gatherings, with a fireplace that crackles with a gentle warmth. The kitchen is a chef's delight, complete with a cauldron-compatible stove and ample space for potion-

## Component 5: Interactive User Interface

HomeMatch's user interface brings the magical real estate experience to life, creating an engaging and interactive platform for wizards to find their dream homes.

### Key Features

1. **Chatbot Interface**: A conversational AI that guides users through the home-searching process, understanding their preferences through natural language.

2. **Persona-Driven Interaction**: The interface features "Mister Sneekrs," a charming magical real estate agent who adds character and whimsy to the experience.

3. **Dynamic Search**: As users chat about their preferences, the system performs real-time searches to find matching properties.

4. **Personalized Listings Display**: Search results are presented in an appealing format, with key details highlighted and personalized descriptions.

5. **Expandable Property Details**: Users can easily view more information about each property through expandable sections.

### Technology

Built using Streamlit, this GUI combines the power of our backend components (LLM, vector store, semantic search) with a user-friendly frontend, creating a seamless and magical house-hunting experience.